In [63]:
import pandas as pd
import numpy as np
np.random.seed(100)

In [64]:
data = pd.read_csv("원본.csv",index_col=0)
data["선거 종류"] = data["선거 종류"].apply(lambda x: x.replace("·","").replace(" ",""))
data["당선여부"] = data["당선여부"].astype(bool)
data["경력_지역"] = data["경력_지역"].astype(bool)

원본 데이터 컬럼 및 데이터 형

영어|컬럼|type
--|--|--
Name|성명|String
Loccation|지역|String
ElectionDistrict|선거구명 | String
ElectionKind|선거 종류 | String
Num | 회차 | Int
SymbolNumber| 기호 | Int
Gender|성별 | String
Party|정당명 | String
Age|연령 | Int
Job|직업 | String
Rater|여당평가 | Float
Elected|당선여부 | Bool       
Vote|득표수(득표율) | Float
School|학교 | String
Subject|과목 | String
Career|경력_지역 | Bool
Popularity|인지도 | Float
LocNum|지역구분 | Int

각 원본데이터의 컬럼 pair를 나타내며, ""는 사라진 컬럼을 의미함  
기본적으로 사라진 컬럼은 Null로 넣으면 되지만 선거종류의 경우 원본데이터 제목에 명시돼있으므로 자동으로 넣는다.

In [65]:
# 매핑 정보
원본 = ['성명', '지역', '선거구명', '선거 종류', '회차', '기호', '성별', '정당명', '연령', '직업',
      '여당평가', '당선여부', '득표수(득표율)', '학교', '과목', '경력_지역', '인지도', '지역구분']
국회의원 = ['이름', '선거구', '선거구명', '종류', '회', '기호', '남여', '정당', '연령', '직업',
        '여당평가', '당선여부', '득표율',      '학교', '과목', ""        , '인지', '지역구분']
시도지사선거 = ['성명', '지역', '선거구명', '선거종류', '', '기호', '', '정당명', '연령', '직업',
      '여당평가', '당선여부', '득표수', '학력', '', '', '인지도', '지역구분']
구시군의장선거 = ['이름', '선거구명1', '선거구명2', '종류', '회', '기호', '남여', '정당', '연령', '직업',
           '여당평가', '당선여부', '득표율',      '', '', ""        , '', '지역구분']
시도의회의원선거 = ['성명', '선거구1', '선거구2', '', '', '', '', '', '', '',
            '여당평가', '당선여부', '득표수', '학력', '과목', '경력_지역', '인지도', '지역구분']
구시군의회의원선거 = ['이름', '위치', '선거구', '', '회', '몇번', 'gender', '정당', '나이', 'job',
      '당시여당평가', '당선', '득표율', '학력', '과', '경력', '인지', '지역구분']

원본데이터 변경사항-> 각 원본 데이터 마다 여러개의 데이터가 존재함 -> 1번부터 2번까지 순서대로 등록 

데이터이름|기본 행개수|all NULL행 추가 row|1번 제외 전체 value NULL 비율|데이터 내 중복 개수|데이터간 중복열 추가
--|--|--|--|--|--
국회의원1|1000|1%|2%|146개|2데이터 100개
국회의원2|1929||8%|98개|
시도지사선거1|183||4%|13개|
구시군의장선거1|500||2%|190개|
구시군의장선거2|1000||1%|145개|1 데이터 100개
구시군의장선거3|680||4%|349개|1,2데이터 100,100개
시도의회의원선거1|1200|2%|5%|28개|
시도의회의원선거2|1200||7%|1000개|3번 데이터 1000개
시도의회의원선거3|2352||3%|13개|
구시군의회의원선거1|5471|3%|5%|2600개|
구시군의회의원선거2|5471||6%|10개|
구시군의회의원선거3|5471|10%|2%||1번, 2번 데이터 각 2500개


In [66]:
names = ['국회의원1', '국회의원2', '시도지사선거1', '구시군의장선거1','구시군의장선거2','구시군의장선거3',
 '시도의회의원선거1','시도의회의원선거2','시도의회의원선거3', '구시군의회의원선거1', '구시군의회의원선거2', '구시군의회의원선거3']
cols = [국회의원, 국회의원, 시도지사선거, 구시군의장선거, 구시군의장선거, 구시군의장선거,
       시도의회의원선거,시도의회의원선거,시도의회의원선거, 구시군의회의원선거, 구시군의회의원선거, 구시군의회의원선거]
rows = [(0,1000), (1000,2929),(0,183), (0,500), (500,1500), (1500,2180), (0,1200), (1200, 2400), (2400, 4752),
        (0,5471), (5471, 10942), (10942, 16413)]
null_rows = [0.01, 0,0,0,0,0,0.02,0,0,0.03,0,0.1]
null_vals = [0.02, 0.08, 0.04, 0.02, 0.01, 0.04, 0.05, 0.07, 0.03, 0.05, 0.06, 0.02]
dup1s = [146,98,13,190,145,349,28,1000,13,2600,10,0]
dup2s = [[(1,100)],None,None,None,[(3,100)],[(3,100),(4,100)],None,[(8,1000)],None,None,None,[(9,2500),(10,2500)]]
tmp_dfs = []
for i in range(12):
    origin = data[data["선거 종류"] == names[i][:-1]].reset_index(drop=True)
    origin = origin[rows[i][0]:rows[i][1]].reset_index(drop=True)
    tmp = origin.copy()
    na = pd.DataFrame( columns = tmp.columns,index = range(len(origin),int(len(origin) + len(origin) * null_rows[11])))
    if len(na) != 0:
        tmp = pd.concat([tmp,na]).reset_index(drop=True)
    for col in tmp.columns:
        tmp.loc[np.random.randint(0,len(origin),(round(origin.shape[0] * null_vals[i]),)),col] = np.nan
    tmp = pd.concat([tmp,origin.loc[np.random.randint(0,len(origin),dup1s[i])]]).reset_index(drop=True)
    if dup2s[i] != None:
        for dup in dup2s[i]:
            tmp_origin = data[data["선거 종류"] == names[dup[0]][:-1]].reset_index(drop=True)
            tmp_origin = tmp_origin.loc[np.random.randint(0,len(tmp_origin),dup[1])]
            tmp = pd.concat([tmp,tmp_origin]).reset_index(drop=True)
    tmp_dfs.append(tmp)
for i in range(12):
    tmp_dfs[i].columns = cols[i]
    tmp_dfs[i] = tmp_dfs[i].drop("",axis=1)  
    tmp_dfs[i].to_csv(names[i] + ".csv", index = False)

In [67]:
for i in range(12):
    dic = dict(zip(cols[i],원본))
    del dic[""]
    tmp_dfs[i] = tmp_dfs[i].rename(columns = dic)
    if 6 <= i <= 8:
        tmp_dfs[i]["선거 종류"] = "시도의회의원선거"
    elif 9 <= i <= 11:
        tmp_dfs[i]["선거 종류"] = "구시군의회의원선거"

In [68]:
result = pd.concat(tmp_dfs)
result = result[result.notna().sum(axis=1) !=0]
result = result[~result.duplicated()]
result = result.reset_index()

C:\Users\ljw56\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [76]:
data = result
a = data.where(pd.notnull(data), "")

In [89]:
a.where(a != "")["과목"][a.where(a != "")["과목"].fillna("").str.contains("법")]

0        법
1        법
7        법
10       법
20       법
        ..
30015    법
30087    법
30116    법
30166    법
30171    법
Name: 과목, Length: 1632, dtype: object

In [70]:
# result["회차"] = result["회차"].dropna().astype(int)
tmp = result["경력_지역"].astype(float)
